In [2]:
import json

import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from pprint import pprint

import plotly.io as pio
pio.renderers.default='jupyterlab'  # notebook doesn't work check https://plotly.com/python/renderers/
pd.options.display.max_columns = 100
pd.options.display.max_rows = 300

In [3]:
mats = pd.read_csv("in/tables/MATERIALS_FOR_CLASSIFICATION_CLEAN.csv")
cats = pd.read_csv("in/tables/MATERIAL_CATEGORY_FOR_CLASSIFICATION.csv")

df = pd.merge(mats,cats,on=["MATERIAL_ODS_ID","SHOP"])

def create_category_dict(df):
    return df.groupby("CATEGORY_ID").CATEGORY_PATH.unique().to_dict()
category_dict = create_category_dict(df)

# should be applied after category_decoder_dict
df = df[~df.MATERIAL_ID.duplicated()].reset_index(drop=True).copy()

for k,v in category_dict.items():
    df.loc[df.CATEGORY_ID == k,"CATEGORY_PATH_FIX"] = v[0]

In [4]:
subset = df.copy()

In [5]:
subset.CATEGORY_PATH = subset.CATEGORY_PATH.str.split(" > ").apply(lambda x: " > ".join(x[1:]))

In [6]:
from sklearn.preprocessing import LabelEncoder

# encode matcat names as integer values because transformer models do not accept strings
subset['CATEGORY_SEQ_TOKENS'] = subset['CATEGORY_PATH'].str.split(" > ")
unique_tokens = []
for i in subset['CATEGORY_SEQ_TOKENS']:
    unique_tokens.extend(i)
    
label_enc = LabelEncoder()
labeled_tokens = label_enc.fit_transform(unique_tokens)

tokens_seq = []
pos=0
for i in subset['CATEGORY_SEQ_TOKENS']:
    tokens_seq.append(labeled_tokens[pos:pos+len(i)])
    pos+=len(i)
    
# new from transformers
int_tokens_seq = [str(i).strip("[ ]").split(" ") for i in tokens_seq]

In [7]:
for i in int_tokens_seq:
    try:
        for _ in range(10): del i[i.index("")]
    except:
        continue

In [8]:
subset['CATEGORY_SEQ'] = tokens_seq

In [9]:
subset["CATEGORY_SEQ_STR"] = subset.CATEGORY_SEQ.apply(lambda x: " ".join(x.astype(str)))

In [10]:
def prepare_MT_data(row):
    return row["NAME_BRIEF"] + " \t " + row["CATEGORY_SEQ_STR"]

In [11]:
subset["text"] = subset.apply(prepare_MT_data, axis=1)

In [12]:
# test_categories = df.CATEGORY_ID.value_counts().reset_index().query("CATEGORY_ID > 20")["index"].values
# df = df.query("CATEGORY_ID in @test_categories").copy()

def remove_too_small_categories(df, how_many=20):
    """
    Remove all categories small than @how_many, 
    because it is unsufficient to add them to classification.
    These categories need to be evaluated in final testing.
    """
    too_small_cats = df.CATEGORY_ID.value_counts().reset_index().query("CATEGORY_ID<@how_many")["index"].unique()
    indices = df[df.CATEGORY_ID.isin(too_small_cats)].index
    small_cat_df = df.iloc[indices].copy()
    print(f"Len of df before: {df.shape[0]}")
    df.drop(indices, inplace=True)
    print(f"Len of df after: {df.shape[0]}")
    return small_cat_df

too_small_categories_df = remove_too_small_categories(subset)



Len of df before: 242110
Len of df after: 240998


In [16]:
from sklearn.model_selection import train_test_split

def create_datasets(df, stratify_by="CATEGORY_ID",make_valid=None, random_state=1):
    """
    Create train/test, possibly also validation dataset.
    If make_valid=True, then returns df,list(df,df)
    """
    train, test = train_test_split(df.reset_index(drop=True), 
                                   stratify=df[stratify_by],
                                   test_size=0.3, 
                                   random_state=random_state)
    if make_valid:
        test = train_test_split(test.reset_index(drop=True),
                                test_size=0.3,
                                random_state=random_state)
    
    return train, test

def over_sample(df, categories_smaller=100):
    """
    Oversample for really small categories
    """
    new_df = pd.DataFrame()
    # smaller categories
    small_categories = (df.CATEGORY_ID
                        .value_counts()
                        .reset_index()
                        .query("CATEGORY_ID < 100"))
    
    for _,cat_id, count in small_categories.itertuples():
        append_another = categories_smaller-count
        df_cat = df.query("CATEGORY_ID == @cat_id")
        new_df = new_df.append(df_cat.iloc[np.random.randint(df_cat.shape[0],size=append_another)])
    
    print(f"Appending {new_df.shape[0]} of new samples")
    return pd.concat([df,new_df])



In [17]:
subset = subset[["MATERIAL_ID","NAME_BRIEF","text","CATEGORY_SEQ", "CATEGORY_PATH", "CATEGORY_ID"]].copy()
# subset = subset[subset.NAME.str.len() >9].copy().reset_index()
# subset.NAME = subset.NAME.str.lower()
# subset.NAME = subset.NAME.str.replace('\n', '')

In [18]:

train, test = create_datasets(subset)
test, validation = create_datasets(test)

In [19]:
train.shape, test.shape, validation.shape

((168698, 6), (50610, 6), (21690, 6))

In [20]:
import csv
def save_set(subset,name):
    subset[["text"]].to_csv(f'data/mt/categorizer.cs_cat.{name}.tsv', index=False, sep="$", header=False,
                           quoting = csv.QUOTE_NONE, escapechar = ' ')
    subset[["NAME_BRIEF"]].to_csv(f'data/mt/categorizer.cs_cat.{name}.tok.cs', index=False, sep="$", header=False,
                           quoting = csv.QUOTE_NONE, escapechar = ' ')
    subset[["CATEGORY_SEQ"]].to_csv(f'data/mt/categorizer.cs_cat.{name}.tok.cat', index=False, sep="$", header=False,
                           quoting = csv.QUOTE_NONE, escapechar = ' ')

In [21]:
!mkdir -p data/mt

In [24]:
save_set(train, "train")
save_set(test, "test")
save_set(validation, "valid")

In [25]:
!fairseq-preprocess \
    --source-lang cs \
    --target-lang cat \
    --trainpref data/mt/categorizer.cs_cat.train.tok \
    --validpref data/mt/categorizer.cs_cat.valid.tok \
    --testpref data/mt/categorizer.cs_cat.test.tok \
    --tokenizer moses \
    --destdir data/mt-bin \
    --thresholdsrc 3 \
    --thresholdtgt 3

2022-01-04 15:10:53 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data/mt-bin', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='cs', srcdict=None, target_lang='cat', task='translation', tensorboard_logdir=None, testpref='data/mt/categorizer.cs_cat.test.tok', tgtdict=None, threshold_loss_scale=None, thresholdsrc=3, thresholdtgt=3, tokenizer='m

In [54]:
!fairseq-train \
    data/mt-bin \
    --arch transformer \
    --optimizer adam \
    --adam-betas '(0.9, 0.98)' \
    --clip-norm 0.0 \
    --lr 5e-4 \
    --tokenizer moses \
    --lr-scheduler inverse_sqrt \
    --dropout 0.2 \
    --criterion label_smoothed_cross_entropy \
    --label-smoothing 0.1 \
    --max-tokens 4096 \
    --max-epoch 30 \
    --save-dir data/mt-ckpt-transformer \
    --encoder-layers 1 \
    --encoder-ffn-embed-dim 512 \
    --decoder-layers 1 \
    --decoder-ffn-embed-dim 512 \
    --tensorboard-logdir data/mt-tensorboard

2022-01-04 17:30:40 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data/mt-bin', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=512, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=1, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_b

# Validation

In [55]:
from fairseq.models.transformer import TransformerModel
model = TransformerModel.from_pretrained(model_name_or_path='/data/data',
                                  checkpoint_file='/data/data/mt-ckpt-transformer/checkpoint_best.pt',
                                         data_name_or_path='/data/data/mt-bin',
)

In [56]:
test_example = model.translate(test.NAME_BRIEF.values, beam=1,verbose=True)

/home/default/lib/python3.8/site-packages/fairseq/search.py:140: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').

/home/default/lib/python3.8/site-packages/fairseq/sequence_generator.py:651: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').



In [57]:
predictions = []
seq = []
for i in test_example:
    sequence = i.split("]")[0].replace("[","").replace("]","").split()
    prediction = label_enc.inverse_transform([int(j) for j in sequence])
    
    seq.append(sequence)
    predictions.append(prediction)

In [58]:
seq[:10]

[['528', '387', '355', '433'],
 ['90', '577'],
 ['327', '426'],
 ['371', '280'],
 ['419', '301', '355'],
 ['334', '257'],
 ['334', '257'],
 ['90', '175', '491'],
 ['523', '201'],
 ['513', '126']]

In [115]:
train.to_csv("train_data_seq2seq.csv",index=False)

In [63]:
test.to_csv("test_data_small_transformer.csv",index=False)

In [116]:
validation.to_csv("valid_data_seq2seq.csv",index=False)

In [59]:
test["predictions"]= [" > ".join(i) for i in predictions]

In [60]:
test["pred_seqs"]= seq

In [61]:
test.query("CATEGORY_PATH != predictions")

,MATERIAL_ID,NAME_BRIEF,text,CATEGORY_SEQ,CATEGORY_PATH,CATEGORY_ID,predictions,pred_seqs
906,100038811702,stropn sušák ; stropn sušák prádl výborn pomoc...,stropn sušák ; stropn sušák prádl výborn pomoc...,"[90, 440]","Dům, byt > Sušáky na prádlo",35,"Dům, byt > Šňůry, sušáky na prádlo","[90, 577]"
37949,100040263921,napínák drátěn plot ocel stříbrn ; tat sad plo...,napínák drátěn plot ocel stříbrn ; tat sad plo...,"[419, 301, 112, 355]",Stavby na zahradě > Ploty > Gabiony > Přísluše...,157759,Stavby na zahradě > Ploty > Příslušenství,"[419, 301, 355]"
53980,100066724714,sprch walk stříbr lesk skl sítotisk ; sprch sa...,sprch walk stříbr lesk skl sítotisk ; sprch sa...,"[90, 174, 546]","Dům, byt > Koupelna a sanitární technika > Zás...",6403678,"Dům, byt > Koupelna, sanitarni technika > Zást...","[90, 175, 546]"
64771,100068350006,ručn vyroben kuchyňsk dřez sítk nerez ocel ; v...,ručn vyroben kuchyňsk dřez sítk nerez ocel ; v...,"[90, 190, 89]","Dům, byt > Kuchyně > Dřezy",1995,"Dům, byt > Koupelna a sanitární technika > Vod...","[90, 174, 491, 88]"
27009,100049525093,matic nýtovac sad ; matic nýtovac sad ocel oce...,matic nýtovac sad ; matic nýtovac sad ocel oce...,"[371, 157, 244]",Ruční nářadí > Kleště > Nýtovací kleště,65221461,Stavební materiál > Spojovací materiál > Nýty,"[422, 404, 245]"
...,...,...,...,...,...,...,...,...
46726,100055743754,nerez roztahovac hadic závit ; nerez roztahova...,nerez roztahovac hadic závit ; nerez roztahova...,"[489, 536, 120]",Voda v zahradě > Zavlažování zahrady > Hadice,337,Voda v zahradě > Zavlažování > Hadice,"[489, 535, 120]"
19705,100030257853,altán krém textil ; tent prostorn zahradn altá...,altán krém textil ; tent prostorn zahradn altá...,"[523, 398, 418]","Zahradní nábytek > Slunečníky, zastínění > Sta...",5603,"Zahradní nábytek > Stany, altany","[523, 418]"
16957,100066834977,temp kondel zahradn altánek modr gotan ; mater...,temp kondel zahradn altánek modr gotan ; mater...,"[419, 66, 23]","Stavby na zahradě > Domky, altány > Altány",5603,"Zahradní nábytek > Stany, altany","[523, 418]"
18371,100059186163,pláž slunečník zelen ; pláž slunečník vyroben ...,pláž slunečník zelen ; pláž slunečník vyroben ...,"[523, 397]",Zahradní nábytek > Slunečníky,220,"Zahradní nábytek > Slunečníky, zastínění > Slu...","[523, 398, 397]"


In [38]:
from sklearn.metrics import f1_score

In [62]:
f1_score(test.CATEGORY_PATH, test.predictions, average="weighted")

0.8132939310529762

In [54]:
label_enc.inverse_transform([int(j) for j in i.strip("[ ]").split()])

array(['Nářadí zahradní', 'Pily'], dtype='<U39')

In [58]:
i.strip("[ ]").split()

['243', '290']

In [47]:
label_enc.inverse_transform([int(i) for i in test_example.strip("[ ]").split()])

AttributeError: 'list' object has no attribute 'strip'

In [37]:
for i in model.generate(model.encode(example)):
    tokens = [int(i) for i in model.decode(i["tokens"]).strip("[ ]").strip("]").split()]
    print(" > ".join(label_enc.inverse_transform(tokens)))

Nářadí zahradní > Pily
Ruční nářadí > Ostatní ruční nářadí
Dílna, stavební technika > Kompresory


ValueError: invalid literal for int() with base 10: '366]'

In [34]:
label_enc.inverse_transform([int(i) for i in test_example.strip("[ ]").split()])

array(['Nářadí zahradní', 'Pily'], dtype='<U39')

# Generate

In [809]:
!fairseq-generate \
    data/mt-bin \
    --path data/mt-ckpt-transformer/checkpoint_best.pt \
    --beam 3 \
    --tokenizer moses \
    --results-path data/mt-test

  0%|                                                     | 0/8 [00:00<?, ?it/s]/home/default/lib/python3.8/site-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/home/default/lib/python3.8/site-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_m

In [778]:
!grep ^T data/mt-test/generate-test.txt | cut -f1- > name.txt

In [779]:
!grep ^T data/mt-test/generate-test.txt | cut -f2- > target.txt

In [780]:
!grep ^H data/mt-test/generate-test.txt | cut -f3- > hypotheses.txt

In [781]:
results = pd.read_csv("name.txt", sep="\t", header=None)

In [782]:
y_pred = pd.read_csv("hypotheses.txt",header=None)
y_pred = y_pred.rename(columns={0:"pred"})

In [783]:
results["index"] = results[0].str.strip("T-")
results = pd.concat((results,y_pred),axis=1)

In [835]:
test = pd.read_csv("data/mt/categorizer.cs_cat.test.tsv",sep="\t", header=None)

In [843]:
test = test.rename(columns={0:"NAME",1:"CATEGORY_SEQ_STR"})

In [852]:
test.NAME.values

array([' sprchový  kout  obdélníkový  kndj2/priii-80x110,  manhatan,  sklo  čiré  600-073-0270-11-401  ',
       ' spony  hadicové,  balení  10ks,  10-16mm  ',
       ' zahradní  zavlažovací  sada  dedra  nts  professional  1/2  "x20m,  5ks.  -  80n741z  ',
       ...,
       " vodovodní  baterie  vanová,  barva:  chrom,  rozměr:  1/2''  (d470.5s)  ",
       ' zahradní  barový  stůl  černý  70  x  70  x  110  cm  polyratan  a  sklo  ',
       ' triple  sib  1p,  komín  zděný,  8  m,  dn180,  90°  '],
      dtype=object)

In [857]:
test.CATEGORY_SEQ_STR = test.CATEGORY_SEQ_STR.apply(lambda x: " ".join(x.split()))
test.NAME = test.NAME.apply(lambda x: " ".join(x.split()))

In [863]:
df = pd.merge(subset,test, on=["NAME","CATEGORY_SEQ_STR"])

In [837]:
evaluation = test.copy()

In [838]:
new_df = pd.DataFrame()
for _,i in results.iterrows():
    index = int(i["index"])
    string_categories = i["pred"].strip("[]").split(" ")[1:]
    categories = [int(i) for i in string_categories]
    encoded_categories = label_enc.inverse_transform(categories)
    
    new = evaluation.iloc[index:index+1].copy()
    new["results"] = [encoded_categories]
    true_categories = label_enc.inverse_transform((new.CATEGORY_SEQ.values[0]))
    new["CATEGORY_TRUE"] = [true_categories]
    size = (true_categories.shape if encoded_categories.shape >= true_categories.shape else encoded_categories.shape)[0]
    new["same"] = (encoded_categories[:size] == true_categories[:size]).all()
    new_df = new_df.append(new)

AttributeError: 'DataFrame' object has no attribute 'CATEGORY_SEQ'

In [288]:
subset["CATEGORY_TRUE"] = subset.CATEGORY_SEQ.apply(label_enc.inverse_transform)

In [306]:
new_df["results_post"] = new_df.results.apply(lambda x: " > ".join(x))

In [311]:
new_cats = new_df["results_post"].unique()
old_cats = subset.CATEGORY_PATH.unique()

In [321]:
for i in new_cats:
    if i in old_cats:
        pass
    else:
        print(i)
        print(new_df[(new_df.results_post == i)][["NAME","CATEGORY_TRUE"]].values)

Hobby a zahrada > Nářadí elektrické > Zahradní houpačky, houpací sítě
[['Diamond flexibilní brusný kotouč'
  array(['Hobby a zahrada', 'Nářadí elektrické', 'Příslušenství', 'Kotouče'],
        dtype='<U31')                                                        ]
 ['Diamond flexibilní brusný kotouč'
  array(['Hobby a zahrada', 'Nářadí elektrické', 'Příslušenství', 'Kotouče'],
        dtype='<U31')                                                        ]]
Hobby a zahrada > Kuchyně > Dřezy
[['Zorba 440E - Sand'
  array(['Hobby a zahrada', 'Dům, byt', 'Kuchyně', 'Dřezy'], dtype='<U31')]]
Hobby a zahrada > Stavby na zahradě > Příslušenství
[['Roleta RÁKOS NATUR 120 x 180 cm'
  array(['Hobby a zahrada', 'Stavby na zahradě', 'Ploty',
         'Zastínění oplocení'], dtype='<U31')            ]]
Hobby a zahrada > Dům, byt > Koupelna a sanitární technika > Vany, sprchy > Vany
[['Shower select baterie pod omítku pro 2 spotřebiče, chrom (15748000)'
  array(['Hobby a zahrada', 'Dům, byt', 'Koupelna

In [285]:
df.CATEGORY_PATH.str.startswith("Hobby a zahrada > Ruční nářadí").sum()

37151

In [280]:
df.CATEGORY_PATH.str.startswith("Hobby a zahrada > Nářadí ruční").sum()

15651

In [385]:
new_df[new_df.same == False]

,index,MATERIAL_ID,NAME,text,CATEGORY_SEQ,results,CATEGORY_TRUE,same,results_post
1808,1808,100059706303,Dvouramenný stahovák ložisek 40mm x 80mm,Dvouramenný stahovák ložisek 40mm x 80mm \t 5 ...,"[5, 2, 45]","[Hobby a zahrada, Ohřev vody, vytápění, Komíny]","[Hobby a zahrada, Dům, byt, Vypínače, zásuvky]",False,"Hobby a zahrada > Ohřev vody, vytápění > Komíny"
387,387,100053067840,"Sada samocentrovacích sklíčidel 3,75 palce, ocel","Sada samocentrovacích sklíčidel 3,75 palce, oc...","[5, 16, 28]","[Hobby a zahrada, Ruční nářadí, Příslušenství]","[Hobby a zahrada, Nářadí ruční, Příslušenství]",False,Hobby a zahrada > Ruční nářadí > Příslušenství
27442,27442,100051464792,Nůž zavírací COBRA 20cm s pojistkou,Nůž zavírací COBRA 20cm s pojistkou \t 5 31 17,"[5, 31, 17]","[Hobby a zahrada, Nářadí ruční, Nůžky, nože, p...","[Hobby a zahrada, Ruční nářadí, Nůžky, nože, p...",False,"Hobby a zahrada > Nářadí ruční > Nůžky, nože, ..."
8422,8422,3728926,FDN 9301 Sponky pro FDN 3001,FDN 9301 Sponky pro FDN 3001 \t 5 31 28,"[5, 31, 28]","[Hobby a zahrada, Nářadí ruční, Příslušenství]","[Hobby a zahrada, Ruční nářadí, Příslušenství]",False,Hobby a zahrada > Nářadí ruční > Příslušenství
23749,23749,100053130855,"mřížka opékací hranatá 33x29cm, délka 62cm","mřížka opékací hranatá 33x29cm, délka 62cm \t ...","[5, 3, 28]","[Hobby a zahrada, Ohřev vody, vytápění, Kamna,...","[Hobby a zahrada, Grily, udírny a kotlíky, Pří...",False,"Hobby a zahrada > Ohřev vody, vytápění > Kamna..."
...,...,...,...,...,...,...,...,...,...
4910,4910,100060409818,"Bity RIBE, různé velikosti, úchyt 10 mm, délka...","Bity RIBE, různé velikosti, úchyt 10 mm, délka...","[5, 31, 28]","[Hobby a zahrada, Stavební materiál, Spojovací...","[Hobby a zahrada, Ruční nářadí, Příslušenství]",False,Hobby a zahrada > Stavební materiál > Spojovac...
4909,4909,100060409818,"Bity RIBE, různé velikosti, úchyt 10 mm, délka...","Bity RIBE, různé velikosti, úchyt 10 mm, délka...","[5, 16, 28]","[Hobby a zahrada, Stavební materiál, Spojovací...","[Hobby a zahrada, Nářadí ruční, Příslušenství]",False,Hobby a zahrada > Stavební materiál > Spojovac...
27683,27683,100060300173,"Bity RIBE, různé velikosti, úchyt 10 mm, délka...","Bity RIBE, různé velikosti, úchyt 10 mm, délka...","[5, 16, 28]","[Hobby a zahrada, Stavební materiál, Spojovací...","[Hobby a zahrada, Nářadí ruční, Příslušenství]",False,Hobby a zahrada > Stavební materiál > Spojovac...
20941,20941,100060300111,"Bity RIBE, různé velikosti, úchyt 10 mm, délka...","Bity RIBE, různé velikosti, úchyt 10 mm, délka...","[5, 31, 28]","[Hobby a zahrada, Stavební materiál, Spojovací...","[Hobby a zahrada, Ruční nářadí, Příslušenství]",False,Hobby a zahrada > Stavební materiál > Spojovac...
